In [ ]:
!pip install tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 3.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot

class DefaultBNQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
        return []

    def get_activations_and_quantizers(self, layer):
        return []

    def set_quantize_weights(self, layer, quantize_weights):
        pass

    def set_quantize_activations(self, layer, quantize_activations):
        pass

    def get_output_quantizers(self, layer):
        return []

    def get_config(self):
        return {}

# Load and prepare data
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

with tf.keras.utils.custom_object_scope({'DefaultBNQuantizeConfig': DefaultBNQuantizeConfig}):
    # Model architecture with quantization-aware training
    input_layer = keras.layers.Input(shape=(28, 28))
    x = keras.layers.Flatten()(input_layer)
    x = keras.layers.Dense(1024, activation='relu')(x)
    x = tfmot.quantization.keras.quantize_annotate_layer(
        keras.layers.BatchNormalization(), DefaultBNQuantizeConfig())(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(1024, activation='relu')(x)
    x = tfmot.quantization.keras.quantize_annotate_layer(
        keras.layers.BatchNormalization(), DefaultBNQuantizeConfig())(x)
    x = keras.layers.Dropout(0.5)(x)
    output_layer = keras.layers.Dense(10, activation='softmax')(x)
    model = keras.Model(inputs=input_layer, outputs=output_layer)

    # Apply quantization to the model
    quantize_model = tfmot.quantization.keras.quantize_apply(model)

# Compile and train
quantize_model.compile(optimizer='adam',
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])
quantize_model.fit(train_images, train_labels, epochs=10, validation_split=0.1)

# Evaluate the model
quantize_model.evaluate(test_images, test_labels)


11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/10
1688/1688 [==============================] - 77s 34ms/step - loss: 0.3684 - accuracy: 0.8939 - val_loss: 0.1046 - val_accuracy: 0.9702
Epoch 2/10
1688/1688 [==============================] - 58s 34ms/step - loss: 0.2118 - accuracy: 0.9364 - val_loss: 0.1080 - val_accuracy: 0.9723
Epoch 3/10
1688/1688 [==============================] - 57s 34ms/step - loss: 0.1840 - accuracy: 0.9444 - val_loss: 0.0934 - val_accuracy: 0.9727
Epoch 4/10
1688/1688 [==============================] - 57s 34ms/step - loss: 0.1606 - accuracy: 0.9516 - val_loss: 0.0756 - val_accuracy: 0.9800
Epoch 5/10
1688/1688 [==============================] - 57s 34ms/step - loss: 0.1495 - accuracy: 0.9543 - val_loss: 0.0770 - val_accuracy: 0.9788
Epoch 6/10
1688/1688 [==============================] - 56s 33ms/step - loss: 0.1378 - accuracy: 0.9577 - val_loss: 0.0711 - val_accuracy: 0.9832
Epoch 7/10
1688/1688 [==============================] - 58s

[0.061024781316518784, 0.9819999933242798]

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import tensorflow_model_optimization as tfmot
from sklearn.metrics import precision_score, recall_score

# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the data
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Prepare labels for precision and recall calculations
test_labels_cat = keras.utils.to_categorical(test_labels)

# Model creation
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define model pruning parameters
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.10,
                                                             final_sparsity=0.80,
                                                             begin_step=2000,
                                                             end_step=6000)
}

# Apply pruning to the whole model
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)

# Compile the pruned model
pruned_model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

# Callback for pruning
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir='/tmp/logs')
]

# Train the pruned model
pruned_model.fit(train_images, train_labels,
                 epochs=10,
                 validation_data=(test_images, test_labels),
                 callbacks=callbacks)

# Evaluate the pruned model
result = pruned_model.evaluate(test_images, test_labels, verbose=0)
predicted_labels = pruned_model.predict(test_images)
predicted_labels = predicted_labels.argmax(axis=1)

# Calculate precision and recall
precision = precision_score(test_labels, predicted_labels, average='macro')
recall = recall_score(test_labels, predicted_labels, average='macro')

print(f"Precision: {precision}, Recall: {recall}")

# Apply quantization
quantize_model = tfmot.quantization.keras.quantize_model
quantized_model = quantize_model(pruned_model)

# Compile the quantized model
quantized_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

# Evaluate the quantized model
quantized_model.evaluate(test_images, test_labels)

# Convert to TensorFlow Lite model with Huffman Coding
converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

# Save the model
with open('compressed_model.tflite', 'wb') as f:
    f.write(tflite_quant_model)

print("Model has been compressed and is ready for deployment.")


Update

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import tensorflow_model_optimization as tfmot
from sklearn.metrics import precision_score, recall_score

# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the data
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Model creation
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define model pruning parameters
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.10,
                                                             final_sparsity=0.80,
                                                             begin_step=2000,
                                                             end_step=6000)
}

# Apply pruning to the whole model
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)

# Train the pruned model
pruned_model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep()
]

pruned_model.fit(train_images, train_labels,
                 epochs=10,
                 validation_data=(test_images, test_labels),
                 callbacks=callbacks)

# Strip the pruning wrappers for quantization compatibility
model_for_quantization = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Quantize the model
with tfmot.quantization.keras.quantize_scope():
    # Apply quantization to the whole model
    quantized_model = tfmot.quantization.keras.quantize_model(model_for_quantization)

quantized_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

# Train and evaluate the quantized model
quantized_model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))
result = quantized_model.evaluate(test_images, test_labels)

# Output precision and recall
predicted_labels = quantized_model.predict(test_images)
predicted_labels = predicted_labels.argmax(axis=1)
precision = precision_score(test_labels, predicted_labels, average='macro')
recall = recall_score(test_labels, predicted_labels, average='macro')

print(f"Precision: {precision}, Recall: {recall}")

# Convert to TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model)
tflite_model = converter.convert()

# Save the model
with open('compressed_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model has been compressed and is ready for deployment.")


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
1875/1875 [==============================] - 46s 8ms/step - loss: 0.2048 - accuracy: 0.9394 - val_loss: 0.1000 - val_accuracy: 0.9691
Epoch 2/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0791 - accuracy: 0.9755 - val_loss: 0.0741 - val_accuracy: 0.9769
Epoch 3/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0523 - accuracy: 0.9850 - val_loss: 0.0561 - val_accuracy: 0.9827
Epoch 4/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0309 - accuracy: 0.9912 - val_loss: 0.0548 - val_accuracy: 0.9833
Epoch 5/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0213 - accuracy: 0.9937 - val_loss: 0.0590 - val_accuracy: 0.9828
Epoch 6/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0156 - accuracy: 0.9957 - val_loss: 0.0563 - val_accuracy: 0.9835
Epoch 7/10
1875/1875 [==============================] - 15s 8ms/s